In [1]:
import os
%pwd

'c:\\Users\\parva\\OneDrive\\Desktop\\MOdel BUilding\\Chicken_Dissease\\chick_Dis_Class\\research'

In [2]:
os.chdir("../") # go back in directory
%pwd

'c:\\Users\\parva\\OneDrive\\Desktop\\MOdel BUilding\\Chicken_Dissease\\chick_Dis_Class'

In [3]:
%pwd

'c:\\Users\\parva\\OneDrive\\Desktop\\MOdel BUilding\\Chicken_Dissease\\chick_Dis_Class'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [8]:
# This class is responsible for setting up callbacks for a deep learning model training process.
# Callbacks like TensorBoard and ModelCheckpoint are used to monitor and save model checkpoints.

class PrepareCallback:

    # The constructor takes a config object, which contains configurations such as log directories and checkpoint file paths.
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    # Property to create a TensorBoard callback.
    # TensorBoard helps in visualizing metrics like loss, accuracy, etc., during training.
    @property
    def _create_tb_callbacks(self):
        # Create a timestamp for uniquely naming log directories for each run.
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        
        # Define the path where the TensorBoard logs will be saved.
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,  # Root directory for TensorBoard logs.
            f"tb_logs_at_{timestamp}",            # Sub-directory with the current timestamp.
        )
        
        # Return a TensorBoard callback, which will write logs to the specified directory.
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    # Property to create a ModelCheckpoint callback.
    # ModelCheckpoint saves the model during training, only saving the best version of the model (based on validation metrics).
    @property
    def _create_ckpt_callbacks(self):
        # Ensure that the filepath is passed as a string
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),  # Convert Path to string
            save_best_only=True
        )

    # Method to get both the TensorBoard and ModelCheckpoint callbacks.
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,  # TensorBoard callback.
            self._create_ckpt_callbacks # ModelCheckpoint callback.
        ]


In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-09-22 22:58:36,550: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-22 22:58:36,550: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-22 22:58:36,550: INFO: common: created directory at: artifacts]
[2024-09-22 22:58:36,550: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-09-22 22:58:36,550: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
